In [46]:
import torch
import torch.nn.functional as F
from PIL import Image
import numpy as np
import tempfile
from resnet import * 

In [47]:
import os
import sys
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [48]:
from scripts.tranform import * 

In [ ]:
# 加载模型字典
model_state_dict_path = "/root/paperwithcode/第三周-训练篇/malware_classification/trained_models/malware_classification%resnet34%best.pt" # 要使用的模型字典地址
model_state_dict = torch.load(model_state_dict_path)
model = ResNet(num_block=[3, 4, 6, 3]).cuda()
model.load_state_dict(model_state_dict)
model.eval()

In [71]:
def predict(data_or_path, mode='bmp'): # support bmp, pe, bytes, raw types
    try: 
        if mode == "bmp":
            img = Image.open(data_or_path)
            img = img.resize((64, 64))
            img_mat = np.asarray(img, dtype=np.float32)
            img_mat = np.reshape(img_mat, (1, 64, 64))
            img_mat = np.repeat(img_mat, 3, axis=0)
            img_mat = torch.from_numpy(img_mat)
            img_mat = torch.unsqueeze(img_mat, axis=0).cuda()
            y_pred = model(img_mat)
            return [t.item() for t in F.softmax(y_pred, 1)[0]]
        if mode == "pe":
            _, bmp_path = tempfile.mkstemp(suffix='.bmp')
            pe2bmp(data_or_path, bmp_path)
            return predict(bmp_path, mode='bmp')
        if mode == "bytes":
            _, bmp_path = tempfile.mkstemp(suffix='.bmp')
            bytes2bmp(data_or_path, bmp_path)
            return predict(bmp_path, mode='bmp')
        else: # mode == "raw torch tensor 3x64x64"
            y_pred = model(data_or_path)
            return [t.item() for t in F.softmax(y_pred, 1)[0]]
    except:
        pass
    return [1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1] # 用于表示缺省值，上述代码中，进行格式转换时可能会出现异常，所以缺省值是必要的

In [72]:
predict("/share/mal/tmp/d5687b5c5cec11c851e84a1d40af3ef52607575487a70224f63458c24481076c",  mode='pe')

[0.6112348437309265,
 0.1269327700138092,
 8.482926205033436e-05,
 0.009858082048594952,
 0.006743908394128084,
 0.0421990230679512,
 0.0006651264848187566,
 0.19975799322128296,
 0.002523412462323904]